# Calculate variable importance for the AR18_18 models
### Fit RF and NN to same subset of data and calculate permutation importance

In [ ]:
### Import relevant packages
# Matrix/vector handling
import numpy as np
import pandas as pd

# Scikit-learn and RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
# Permutation importance
from sklearn.inspection import permutation_importance

# Colinearity
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from collections import defaultdict

# NN
from lib import NeuralNet
from keras.wrappers.scikit_learn import KerasClassifier

# Plotting
import matplotlib.pyplot as plt

### Load Data

In [ ]:
### Initialize Onehotencoder and Scaler
onehotencoder = OneHotEncoder(categories="auto")
sc = StandardScaler()

### Read confusion matrices, one hot-encoded (for NN), one not (decision trees)
X_cat = pd.read_pickle("./DataFiles/FeatureMatrixCats.pkl")
X_dum = pd.read_pickle("./DataFiles/FeatureMatrixDummy.pkl")

# Read target
y_cat = pd.read_pickle("./DataFiles/TargetVTs.pkl")
y_dum = LabelBinarizer().fit_transform(y_cat)

### Normalize the NN feature matrix
X_dum_scaled = np.array(X_dum)
X_dum_scaled[:,3:65] = sc.fit_transform(X_dum_scaled[:,3:65]) # Omit x/y/plotid and cat. dummy variables

### Construct final feature matrices

# Remove variables from data frames that should not be in feature matrix
rmvars = ["x","y","plot_id","geology_norge1"]
X_rf = X_cat.drop(rmvars, axis=1)

# Remove from scaled numerical matrix as well, mind different col names for cat. variables!
rmvars = ["x","y","plot_id","geology_norge1_1","geology_norge1_2","geology_norge1_3"]
idx_rm = [X_dum.columns.get_loc(c) for c in rmvars if c in X_dum]
X_nn = np.delete(X_dum_scaled, idx_rm, axis=1)


print(X_rf.columns)

### TEST COLLINEARITY BEFORE SUBSETTING

In [ ]:
#import seaborn as sns
fig, ax = plt.subplots(figsize=(24, 20))
corr = X_rf.corr(method = 'spearman')
sns.set(font_scale=4)
cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=30, as_cmap=True)

b = sns.heatmap(corr, annot = False, cmap=cmap)

#b.axes.set_title("Title",fontsize=50)
#b.set_xlabel("X Label",fontsize=30)
#b.set_ylabel("Y Label",fontsize=20)

#b.tick_params([])#labelsize=12)
fig.tight_layout()
plt.savefig('./Results/FigureFiles/FeatureSpearmanCorr.png')
plt.show()

In [ ]:
### Test collinearity
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))
corr = spearmanr(X_rf).correlation
corr_linkage = hierarchy.ward(corr)
dendro = hierarchy.dendrogram(
    corr_linkage, labels=X_rf.columns.tolist(), ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro['ivl']))

ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
ax2.set_yticklabels(dendro['ivl'])
fig.tight_layout()
#plt.savefig('./Results/FigureFiles/CollinearityInputFeats.png')
plt.show()


In [ ]:
### Use only 1 variable per cluster to reduce importance bias

cluster_ids = hierarchy.fcluster(corr_linkage, 1, criterion='distance')
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
'''
cluster_id_to_feature_ids contains clusters of correlated variables as seperate lists.
Size of lists is determined by "k", the second input of hierarchy.fcluster.
Higher k --> more correlation required to end up in same list
'''
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

X_sel = X_rf.iloc[:, selected_features]
print(X_sel.columns)
print(X_sel.shape)
#clf_sel = RandomForestClassifier(n_estimators=500, random_state=42,n_jobs=-1)
#clf_sel.fit(X_train_sel, y_train)
#print("Accuracy on test data with features removed: {:.2f}".format(
      #clf_sel.score(X_test_sel, y_test)))

In [ ]:
print(X_rf.shape)
print(X_rf.columns)
print(X_onehot.shape)

In [ ]:
### Adjust hot encoded data frame

# Create a second data frame with hot-one encoded categorical variables
X_onehot = X_sel.select_dtypes(exclude=['category'])

X_categories = X_sel.select_dtypes(include=["category"])
categories = X_categories.columns

for cat in categories:
    cur = pd.get_dummies(X_categories[cat],prefix=cat)
    X_onehot = pd.concat(objs=[X_onehot, cur],axis=1)

In [ ]:
X_onehot.info()
cont_idx = [i for i,j in enumerate(X_onehot.columns) if X_onehot[j].dtype!='uint8']

In [ ]:
### Normalize the NN feature matrix
X_nn = np.array(X_onehot)
X_nn[:,cont_idx] = sc.fit_transform(X_nn[:,cont_idx]) # Omit x/y/plotid and cat. dummy variables

In [ ]:
X_nn.shape

### Subset data

In [ ]:
### SUBSET DATA
test_prop = 0.2

# Get random indices to subset data
train_idx = np.random.choice(range(X_nn.shape[0]), int(X_nn.shape[0]*(1-test_prop)), replace=False)
train_idx = sorted(train_idx)
test_idx = [i for i in range(X_nn.shape[0]) if i not in train_idx] 

In [ ]:
### Categorical
X_rf_train = X_sel.iloc[train_idx,:]
y_rf_train = y_cat.iloc[train_idx]
X_rf_test = X_sel.iloc[test_idx,:]
y_rf_test = y_cat.iloc[test_idx]

### One-hot
X_nn_train = X_nn[train_idx,:]
y_nn_train = y_dum[train_idx,:]
X_nn_test = X_nn[test_idx,:]
y_nn_test = y_dum[test_idx,:]


### Initialize the models

In [ ]:
### Function to create the dense MLP
nn = KerasClassifier(build_fn=NeuralNet.initializeNN, verbose=1)

### Scikit Learn
rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=18, max_features='auto',\
                                    bootstrap=True, oob_score=True, verbose=1, n_jobs=-1)

In [ ]:
X_nn_train.shape[1]

In [ ]:
y_nn_train.shape[1]

## Train models

In [ ]:
# Fit NN
history = nn.fit(X_nn_train, y_nn_train, validation_data=(X_nn_test, y_nn_test), epochs=500, verbose=0)

In [ ]:
# Fit RF
rf.fit(X_rf_train, y_rf_train)

In [ ]:
y_predict_nn = nn.predict(X_nn_test)

In [ ]:
imp_nn = permutation_importance(nn, X_nn_test, y_nn_test, n_repeats=5, random_state=7)

In [ ]:
imp_nn_df = pd.DataFrame(columns=['Mean','Std'])
for i in imp_nn.importances_mean.argsort()[::-1]:
    if imp_nn.importances_mean[i] - 2 * imp_nn.importances_std[i] > 0:
        print(f"{X_onehot.columns[i]:<40}" f"{imp_nn.importances_mean[i]:.3f}" f" +/- {imp_nn.importances_std[i]:.3f}")
        imp_nn_df.loc[X_onehot.columns[i],'Mean'] = imp_nn.importances_mean[i]
        imp_nn_df.loc[X_onehot.columns[i],'Std'] = imp_nn.importances_std[i]

In [ ]:
imp_nn_df.to_csv("./Results/Tables/NN_permutation.csv", sep=',',index=True)

In [ ]:
imp_nn_df.__dict__

In [ ]:
imp_rf = permutation_importance(rf, X_rf_test, y_rf_test, n_repeats=5, random_state=7)

In [ ]:
imp_rf_df = pd.DataFrame(columns=['Mean','Std'])
for i in imp_rf.importances_mean.argsort()[::-1]:
    if imp_rf.importances_mean[i] - 2 * imp_rf.importances_std[i] > 0:
        print(f"{X_rf_train.columns[i]:<40}" f"{imp_rf.importances_mean[i]:.3f}" f" +/- {imp_rf.importances_std[i]:.3f}")
        imp_rf_df.loc[X_rf_train.columns[i],'Mean'] = imp_rf.importances_mean[i]
        imp_rf_df.loc[X_rf_train.columns[i],'Std'] = imp_rf.importances_std[i]


#for i in imp_rf.importances_mean.argsort()[::-1]:
#    if imp_rf.importances_mean[i] - 2 * imp_rf.importances_std[i] > 0:
#        print(f"{X_rf_train.columns[i]:<40}" f"{imp_rf.importances_mean[i]:.3f}" f" +/- {imp_rf.importances_std[i]:.3f}")

In [ ]:
imp_rf_df = imp_rf_df.round(3)#.to_csv("./Results/Tables/RF_permutation.csv", sep=',',index=True)
imp_rf_df.round(3)

In [ ]:
imp_rf.importances_mean

In [ ]:
### Compare accuracy
from sklearn.metrics import accuracy_score
y_pred_nn = nn.predict(X_nn_test)
y_pred_rf = rf.predict(X_rf_test)

In [ ]:
acc_nn = nn.score(X_nn_test, y_nn_test)

In [ ]:
acc_rf = accuracy_score(y_pred_rf, y_rf_test)

In [ ]:
print(acc_nn)
print(acc_rf)

In [ ]:
print(imp_rf_df.index[1])

In [ ]:
import importlib
importlib.reload(NeuralNet)

In [ ]:
### Put everything into one data frame and print as latex
permutation_df = pd.DataFrame(columns=['RandomForestClassifier','MultiLayerPerceptron'])
permutation_df.loc['TestAccuracy'] = [acc_rf, acc_nn]

permutation_vars = pd.DataFrame(columns=['RFvarName','RFval','NNvarName','NNval'])
for i in range(5):
    cur_var_rf = imp_rf_df.index[i]
    cur_var_nn = imp_nn_df.index[i]
    permutation_vars.loc[str(i+1)] = [cur_var_rf,
                                      str(np.round(imp_rf_df.loc[cur_var_rf,'Mean'],3))+" (±"+str(np.round(imp_rf_df.loc[cur_var_rf,'Std'],3))+")",
                                      cur_var_nn,
                                      str(np.round(imp_nn_df.loc[cur_var_nn,'Mean'],3))+" (±"+str(np.round(imp_nn_df.loc[cur_var_nn,'Std'],3))+")"]

In [ ]:
X_rf.columns

In [ ]:
#permutation_vars.to_csv('./Results/Tables/VarImpComparison.csv', sep=',',index=True)
#permutation_df.to_csv('./Results/Tables/VarImpAcc.csv', sep=',',index=True)

print(permutation_vars.to_latex(caption="The five most important variables according to permutation importance with n=5 random permutations."))
print(permutation_df.to_latex(caption="The five most important variables according to permutation importance with n=5 random permutations."))